In [1]:
import json, pycm, pandas as pd
from nl_classifier import NaturalLanguageClassifer
from datetime import datetime

In [2]:
MODEL_NAME = 'gpt-4'
TEMPERATURE = 0.1
RESULTS_FILENAME = 'caligraph_gpt-4_serialization_results.json'

In [3]:
experiments = json.load(open(RESULTS_FILENAME, 'r'))

In [4]:
for i, experiment in enumerate(experiments):
    cls = experiment["concept"]
    if "created" in experiment:
        print(f'{i+1:02}: {cls["label"]:30} {cls["id"]} SKIPPING')
    else:
        classifier = NaturalLanguageClassifer(cls["id"], cls["label"], cls["serialization"], MODEL_NAME, TEMPERATURE)
        print(f'{i+1:02}: {cls["label"]:30} {cls["id"]}')
        data = experiment["data"]
        for j, entity in enumerate(data):
            if "predicted" in entity:
                print(f'   {j+1:02}: {entity["label"]:30} {entity["id"]:45} SKIPPING')
            else:
                print(f'   {j+1:02}: {entity["label"]:30} {entity["id"]:45} {entity["actual"]:10} {classifier.tokens_used(entity["label"] + " " + entity["serialization"]):05} tokens ', end=" ")
                classification = classifier.classify(entity["label"], entity["serialization"])
                entity["predicted"] = classification["predicted"].lower()
                if entity["actual"] != entity["predicted"]:
                    if entity["actual"] == "positive":
                        print("FN")
                    else:
                        print("FP")
                else:
                    print("")
                entity["rationale"] = classification["rationale"]
            df_results = pd.DataFrame.from_records(data)
            cm = pycm.ConfusionMatrix(df_results["actual"].tolist(), df_results["predicted"].tolist(), digit=2, classes=[ 'positive', 'negative' ])
            experiment["confusion_matrix"] = cm.matrix
            experiment["created"] = datetime.now().isoformat()
            json.dump(experiments, open(RESULTS_FILENAME, 'w+'))


01: Argentine football manager     http://caligraph.org/ontology/Argentine_football_manager SKIPPING
02: Populated place in Luxembourg  http://caligraph.org/ontology/Populated_place_in_Luxembourg SKIPPING
03: Single-camera television sitcom http://caligraph.org/ontology/Single-camera_television_sitcom
   01: Great News                     http://caligraph.org/resource/Great_News      SKIPPING
   02: Parks and Recreation           http://caligraph.org/resource/Parks_and_Recreation SKIPPING
   03: The Patty Duke Show            http://caligraph.org/resource/The_Patty_Duke_Show SKIPPING
   04: The Smothers Brothers Show     http://caligraph.org/resource/The_Smothers_Brothers_Show SKIPPING
   05: Blandings (TV series)          http://caligraph.org/resource/Blandings_(TV_series) SKIPPING
   06: Clueless (TV series)           http://caligraph.org/resource/Clueless_(TV_series) SKIPPING
   07: Community (TV series)          http://caligraph.org/resource/Community_(TV_series) SKIPPING
   08: Ka